In [ ]:
import matplotlib.pyplot as plt

import findspark
# findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
findspark.init('/home/ubuntu/spark-3.1.1-bin-hadoop3.2')

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Iteration4').getOrCreate()
from pyspark.ml.stat import Correlation

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]
df = spark.createDataFrame(data, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))


In [ ]:
# Reading the datasets
dataset_one = spark.read.csv('./student-math-dataset1.csv', header=True, inferSchema=True)
dataset_two = spark.read.csv('./student-math-dataset2.csv', header=True, inferSchema=True)

In [ ]:
# 2.2 Describe the data
print("Dataset one structure: " + str(dataset_one.count()) + " rows, and " + str(len(dataset_one.columns)) + " columns")
print("Dataset two structure: " + str(dataset_two.count()) + " rows, and " + str(len(dataset_two.columns)) + " columns")
print("How many students inside the dataset: " + str(dataset_one.count()))

In [ ]:
print("Data type inside the dataset one")
dataset_one.dtypes
# print(dataset_one.printSchema())
# print(dataset_two.printSchema())

In [ ]:
print("Data type inside the dataset two")
dataset_two.dtypes

In [ ]:
# 2.3 Explore the data
# need to see the difference between .show() vs .describe().shows()
print("Dataset one information: ")
dataset_one.describe().show() # truncate=False

In [ ]:
dataset_two.describe().show()

In [ ]:
# same as above
# dataset_one.select(dataset_one.columns).describe().show()
# dataset_two.select('G1', 'G2', 'G3').describe().show()

In [ ]:
print("Student first year grade distribution: ")
plt.xlabel('First Year Grade', fontsize = 10)
plt.ylabel('Number of students', fontsize = 10)
plt.title('Student first year grade distribution', fontsize = 15)
diagram_one = dataset_two.toPandas()['G1'].value_counts()
plt.bar(diagram_one.index, diagram_one.values)
plt.show()

In [ ]:
print("Student second year grade distribution: ")
plt.xlabel('Second year Grade', fontsize = 10)
plt.ylabel('Number of students', fontsize = 10)
plt.title('Student second year grade distribution', fontsize = 15)
diagram_two = dataset_two.toPandas()['G2'].value_counts()
plt.bar(diagram_two.index, diagram_two.values)
plt.show()

In [ ]:
print("Student final year grade distribution: ")
plt.xlabel('Final year Grade', fontsize = 10)
plt.ylabel('Number of students', fontsize = 10)
plt.title('Student final year grade distribution', fontsize = 15)
diagram_three = dataset_two.toPandas()['G3'].value_counts()
plt.bar(diagram_three.index, diagram_three.values)
plt.show()

In [ ]:
print("Gender distribution inside the dataset: ")
plt.xlabel('Gender', fontsize = 10)
plt.ylabel('Number of students', fontsize = 10)
plt.title('Gender distribution', fontsize = 15)
diagram_four = dataset_one.toPandas()['sex'].value_counts()
plt.bar(diagram_four.index, diagram_four.values)
plt.show()

print('Number of male students:', len(dataset_one.toPandas()[dataset_one.toPandas()['sex'] == 'M']))
print('Number of female students:', len(dataset_one.toPandas()[dataset_one.toPandas()['sex'] == 'F']))

In [ ]:
print("Student age distribution: ")
plt.xlabel('Age', fontsize = 10)
plt.ylabel('Number of students', fontsize = 10)
plt.title('Student age distribution', fontsize = 15)
diagram_five = dataset_one.toPandas()['age'].value_counts()
plt.bar(diagram_five.index, diagram_five.values)
plt.show()

In [ ]:
print("Student in Urban or rural areas distribution")
plt.xlabel('Area', fontsize = 10)
plt.ylabel('Number of students', fontsize = 10)
plt.title('Student in Urban or rural areas distribution', fontsize = 15)
diagram_six = dataset_one.toPandas()['address'].value_counts()
plt.bar(diagram_six.index, diagram_six.values)
plt.show()

print('Number of students in urban area:', len(dataset_one.toPandas()[dataset_one.toPandas()['address'] == 'U']))
print('Number of students in rural area:', len(dataset_one.toPandas()[dataset_one.toPandas()['address'] == 'R']))

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
# 2.4 Verify the data quality
# missing value from freetime, dalc, walc, health field.
print("Missing value or null value in dataset one: ")
print(dataset_one.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dataset_one.columns]).toPandas().transpose())
print()
print("Missing value or null value in dataset two: ")
print(dataset_two.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dataset_two.columns]).toPandas().transpose())

In [ ]:
# 3.4 integraet various data source
# combine the dataset
dataset_three = dataset_one.join(dataset_two, on=['id'])
print("Dataset one structure: " + str(dataset_three.count()) + " rows, and " + str(len(dataset_three.columns)) + " columns")

In [ ]:
# 2.3 Explore the data（more ploting)
# plot more diagram in this section

In [ ]:
print("Students go out time: ")
diagram_seven_x = dataset_three.toPandas()['goout']
diagram_seven_y = dataset_three.toPandas()['G3']
plt.xlabel('Go out', fontsize = 10)
plt.ylabel('Final Grade G3', fontsize = 10)
plt.title('Students go out time', fontsize = 15)
plt.scatter(diagram_seven_x, diagram_seven_y)
plt.show()

In [ ]:
print("Students study time: ")
diagram_eight_x = dataset_three.toPandas()['studytime']
diagram_eight_y = dataset_three.toPandas()['G3']
plt.xlabel('Study Time', fontsize = 10)
plt.ylabel('Final Grade G3', fontsize = 10)
plt.title('Students study time', fontsize = 15)
plt.scatter(diagram_eight_x, diagram_eight_y)
plt.show()

In [ ]:
print("Mother's job:")
plt.xlabel("Mother's job", fontsize = 10)
plt.ylabel('Counts', fontsize = 10)
plt.title("Mother's job", fontsize = 15)
diagram_nine = dataset_three.toPandas()["Mjob"].value_counts()
plt.bar(diagram_nine.index, diagram_nine.values)
plt.show()

In [ ]:
print("Father's job:")
plt.xlabel("Father's job", fontsize = 10)
plt.ylabel('Counts', fontsize = 10)
plt.title("Mother's job", fontsize = 15)
diagram_ten = dataset_three.toPandas()["Fjob"].value_counts()
plt.bar(diagram_ten.index, diagram_ten.values)
plt.show()

In [ ]:
print("Father's education level:")
plt.xlabel("Father's education level", fontsize = 10)
plt.ylabel('Counts', fontsize = 10)
plt.title("Father's education level", fontsize = 15)
diagram_eleven = dataset_three.toPandas()["Fedu"].value_counts()
plt.bar(diagram_eleven.index, diagram_eleven.values)
plt.show()

In [ ]:
print("Mother's education level:")
plt.xlabel("Mother's education level", fontsize = 10)
plt.ylabel('Final Grade G3', fontsize = 10)
plt.title("Mother's education level", fontsize = 15)
diagram_twelve = dataset_three.toPandas()["Medu"].value_counts()
plt.bar(diagram_twelve.index, diagram_twelve.values)
plt.show()

In [ ]:
# 3.2 Clean the data
# removing the missing value
print("Dataset three length before removing the missing value:", dataset_three.count())
dataset_three = dataset_three.na.drop()
print("Dataset three length after removing the missing value:", dataset_three.count())

In [ ]:
# 3.5 Format the data as required
print("Data type inside the dataset three before formating: ")
dataset_three.dtypes

In [ ]:
# from pyspark.ml.feature import StringIndexer
# indexer = StringIndexer(inputCol="Mjob", outputCol="Mjob_index")
# dataset_three = indexer.fit(dataset_three).transform(dataset_three)
# dataset_three = dataset_three.drop('Mjob')

In [ ]:
types = dict(dataset_three.dtypes)
column_names = []
for key, value in types.items():
    if value == 'string':
        column_names.append(key)
print(column_names)

In [ ]:
from pyspark.ml.feature import StringIndexer
for item in column_names:
    indexer = StringIndexer(inputCol=str(item), outputCol= str(item) + '_index')
    dataset_three = indexer.fit(dataset_three).transform(dataset_three)
for item in column_names:
    dataset_three = dataset_three.drop(item)

In [ ]:
dataset_three.dtypes

In [ ]:
# 3.2 Clean the data (part two)
# removing the outliers value
# import numpy as np
# from scipy import stats
# print("Dataset three length before removing the outliers value:", dataset_three.count())
# dataset_three_z = np.abs(stats.zscore(dataset_three))
# threshold = 5
# dataset_three = dataset_three[(dataset_three_z <= threshold).all(axis=1)]
# print("Dataset three length after removing the outliers value:", dataset_three.count())

In [ ]:
# 4.1 Reduce the data
# Find correlations with the Grade
dataset_three_correlated_fileds = dataset_three.corr()['G3'].sort_values(ascending=False)
print("List out the top 30 fields which has strong correlations relationship with the Final Grade G3: ")
print(dataset_three_correlated_fileds[0:30])

In [ ]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (7, Vectors.dense([0.0, 0.0, 18.0, 1.0]), 1.0,),
    (8, Vectors.dense([0.0, 1.0, 12.0, 0.0]), 0.0,),
    (9, Vectors.dense([1.0, 0.0, 15.0, 0.1]), 0.0,)], ["id", "features", "clicked"])

selector = ChiSqSelector(numTopFeatures=1, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="clicked")

result = selector.fit(df).transform(df)

print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())
result.show()

In [ ]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

selector = ChiSqSelector(numTopFeatures=1, featuresCol="G2",
                         outputCol="selectedFeatures", labelCol="G3")
result = selector.fit(dataset_three).transform(dataset_three)

print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())
result.show()

In [ ]:
# 4.2 Project the data
# Removed
# Will only grabe the top 10 most correlation features for the further study
dataset_three_correlated_fileds = dataset_three_correlated_fileds[:10]
print(dataset_three_correlated_fileds)

dataset_three = dataset_three.loc[:, dataset_three_correlated_fileds.index]
print("\nThe final dataset strucure after cleaning and data transformation:", dataset_three.shape)

In [ ]:
dataset_three = dataset_one.join(dataset_two, on=['id'])

In [ ]:
dataset_three.count()

In [ ]:
# from pyspark.ml.feature import ChiSqSelector
# from pyspark.ml.linalg import Vectors

# df = spark.createDataFrame([
#     (7, Vectors.dense([0.0, 0.0, 18.0, 1.0]), 1.0,),
#     (8, Vectors.dense([0.0, 1.0, 12.0, 0.0]), 0.0,),
#     (9, Vectors.dense([1.0, 0.0, 15.0, 0.1]), 0.0,)], ["id", "features", "clicked"])

# selector = ChiSqSelector(numTopFeatures=1, featuresCol="features",
#                          outputCol="selectedFeatures", labelCol="clicked")

# result = selector.fit(df).transform(df)

# print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())
# result.show()


In [ ]:
# from pyspark.ml.feature import PCA
# from pyspark.ml.linalg import Vectors

# data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
#         (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
#         (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
# df = spark.createDataFrame(data, ["features"])

# pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
# model = pca.fit(df)

# result = model.transform(df).select("pcaFeatures")
# result.show(truncate=False)